# Database operations

As well as accessing and manipulating data and databases, an important part of data science is how we can combine different datasets together.

We can use the `pandas` module to demonstrate different aspects of [combining DataFrames](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html) but these principles are actually more widely applicable to many databases in general (for example compare [pandas operations with SQL](https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_sql.html#compare-with-sql-join)).

## Merging databases

The main function for combining DataFrames in pandas is the [`merge()` function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html). This contains the logic for lots of standard database join operations.

When using this function to combine databases, there are a few things we need to consider about the intended output of the join and the relationship between the databases.

### Set logic

Overall, when deciding how we want to combine databases we need to consider *set logic*. This is the idea around "unions", "intersections" and "differences" of data sets. This can be nicely illustrated using Venn diagrams as shown below:

![Venn diagram of union, intersection and difference](images/Set_logic.png)

A "union" encompasses both data sets, an "intersection" is the overlap between the data sets and a "difference" (as it sounds) is what is different between them.

When using the `merge()` function and otherwise, `pandas` uses the idea of an "inner" join to represent an *intersection* and an "outer" join to represent *union*. You can also use options like the [`df.compare()` method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.compare.html) to look at a *difference* but this will not be covered here.

If we define two DataFrames with some overlapping data, we can see what merging databases would start to look like:

In [25]:
import pandas as pd

# Data set of students and the degree they are studying
students = pd.DataFrame({"name": ["Frankie", "Bill", "Steven", "Juan"],
                         "degree":["Biology", "Mathematics", "Archaeology", "Archaeology"]})
students

,name,degree
0,Frankie,Biology
1,Bill,Mathematics
2,Steven,Archaeology
3,Juan,Archaeology


In [26]:
# Data set of each degree and some associated details
degree_details = pd.DataFrame({"degree":["Astronomy", "Archaeology", "Biology"],
                    "final_year_percentage": ["40", "50", "30"]})
degree_details

,degree,final_year_percentage
0,Astronomy,40
1,Archaeology,50
2,Biology,30


Here we have two DataFrames: one containing student names and the degree they're studying and the other containing information about a set of degrees. We can use the `merge()` function to combine those two DataFrames. In this case the simplest way to do this is to use an overlapping column name, in this case "degree":

In [27]:
pd.merge(students, degree_details, on="degree", how="inner") # default - intersection

,name,degree,final_year_percentage
0,Frankie,Biology,30
1,Steven,Archaeology,50
2,Juan,Archaeology,50


In [28]:
pd.merge(students, degree_details, on="degree", how="outer") # union

,name,degree,final_year_percentage
0,Frankie,Biology,30
1,Bill,Mathematics,NaN
2,Steven,Archaeology,50
3,Juan,Archaeology,50
4,NaN,Astronomy,40


You can see that the `how` keyword can be used to decide how to merge these DataFrames and the difference this makes when using an "inner" (intersection) and "outer" (union) join. What happens when there is a missing entry?

Another option for `how` is to retain all entries for the "left" or "right" DataFrame:

In [29]:
pd.merge(students, degree_details, on="degree", how="left")

,name,degree,final_year_percentage
0,Frankie,Biology,30
1,Bill,Mathematics,NaN
2,Steven,Archaeology,50
3,Juan,Archaeology,50


**Quick question**: What do you think the merged DataFrame will look like if you merge based on the "right" DataFrame (`degree_details`) rather than "left" (`students`)? How many rows would this have? Try this below and see if this is what you expected.

In [30]:
pd.merge(students, degree_details, on="degree", how="right")

,name,degree,final_year_percentage
0,NaN,Astronomy,40
1,Steven,Archaeology,50
2,Juan,Archaeology,50
3,Frankie,Biology,30


---

### Extra: Merge methods

When merging databases it can also be necessary to consider the mapping between them. This can be characterised as:
 
- one-to-one (`one_to_one`)
- one-to-many or many-to-one (`one_to_many` or `many_to_one`)
- many-to-many (`many_to_many`)

This ties to the idea of whether we expect entries within each data set to be unique. For a one-to-one mapping we could expect the mapping column to be unique with one entry per value in *both* tables. For example if we consider another DataFrame defined for degree programmes:

In [31]:
degree_numbers = pd.DataFrame({"degree": ["Mathematics", "History", "Astronomy"],
                               "number_of_students": [321, 157, 56]})

In [32]:
pd.merge(degree_details, degree_numbers, on="degree", how="outer", validate="one_to_one")

,degree,final_year_percentage,number_of_students
0,Astronomy,40,56.0
1,Archaeology,50,NaN
2,Biology,30,NaN
3,Mathematics,NaN,321.0
4,History,NaN,157.0


We used the `validate` keyword here to check that the mapping matched to what we expected. In this case that was `one_to_one`: we expected each entry in the course column to be unique in both data sets.

If we look at another DataFrame, this time defining units and the degree they are each associated with:

In [33]:
degree_units = pd.DataFrame({"unit":["Life Processes", "Microbiology", "Ancient Saxons", "Analysis"],
                        "degree":["Biology", "Biology", "Archaeology", "Mathematics"]})

In this case, we may expect a different mapping if we match to `degree_details`:

In [34]:
pd.merge(degree_details, degree_units, on="degree", how="outer", validate="one_to_many")

,degree,final_year_percentage,unit
0,Astronomy,40,NaN
1,Archaeology,50,Ancient Saxons
2,Biology,30,Life Processes
3,Biology,30,Microbiology
4,Mathematics,NaN,Analysis


We still expect `degree` to be unique in our first data set but don't expect it be unique in the second one. We checked this by validating against a `one_to_many` mapping.

**Extended quick question:** For the case explored above when merging the `students` and `degree_details` DataFrames - which input would best describe the expected mapping?

```
pd.merge(students, degree_details, on="degree", how="left")
```

Using this merge statement, add an additional `validate` input with your expected keyword (e.g. `one_to_one`, `one_to_many`, `many_to_one`, `many_to_many`) to see if this is what you expected (and try a few other options if its not).

In [35]:
pd.merge(students, degree_details, on="degree", how="left", validate = "many_to_one")

,name,degree,final_year_percentage
0,Frankie,Biology,30
1,Bill,Mathematics,NaN
2,Steven,Archaeology,50
3,Juan,Archaeology,50


See the [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html#database-style-dataframe-or-named-series-joining-merging) on database-style merging for more details around the merge function, other functions and how to specify the logic to be used.

---

## Using real data

We can use a modified version of the SQLite database from the first activity to try this out with some real data. The new version, called `ukpvgeo_county.db`, contains new columns for the location of the solar panel in a county or region within the UK (e.g. "Oxfordshire", "Somerset", "Merthyr Tydfil"):

 - "county" (and "county_id")
 
This will allow us to match against other data available on a per county basis. In this exercise, we can match against population data and save our new database with these details.

#### Solar panel database

We can access this SQLite database in a similar way to before by constructing our SQL query to select only columns we are interested in. For a reminder of the available columns within the database, see ["README_ukpvgeo.txt"](data/README_ukpvgeo.txt) (stored in the "data" directory).

*You will find within this database that some details are incomplete but we can see if we can extract any meaningful outputs from these comparisons.*

In [36]:
import pandas as pd

connectable = f"sqlite:///data/ukpvgeo_county.db"
query = "SELECT repd_site_name, capacity_repd_MWp, capacity_osm_MWp, area_sqm, \
                county, county_id \
         FROM pv"

ukpvgeo = pd.read_sql(query, connectable)
ukpvgeo

,repd_site_name,capacity_repd_MWp,capacity_osm_MWp,area_sqm,county,county_id
0,Manor Farm,NaN,NaN,29896.5,Cornwall,E06000052
1,Manor Farm,NaN,NaN,33670.4,Cornwall,E06000052
2,Manor Farm,NaN,NaN,35991.1,Cornwall,E06000052
3,Howton Farm,NaN,NaN,80822.7,Cornwall,E06000052
4,East Langford Farm,5.0,NaN,105138.9,Cornwall,E06000052
...,...,...,...,...,...,...
258781,None,NaN,NaN,0.0,Harrow,E09000015
258782,None,NaN,NaN,0.0,Harrow,E09000015
258783,None,NaN,NaN,0.0,Harrow,E09000015
258784,None,NaN,0.00903,0.0,Harrow,E09000015


#### County population data set

For a population data set split by UK county, this is available from the Office of National Statistics who produce a [yearly population estimate](https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/populationestimatesforukenglandandwalesscotlandandnorthernireland). This data has been downloaded as an Excel file called "ukpopestimatesmid2020on2021geography.xls" (stored within the "data" folder).

We can use pandas tools, [`read_excel()`](https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html), to access this data. Here we have chosen the Excel sheet and the columns we're interested in:

In [40]:
import pandas as pd

population_filename = "data/ukpopestimatesmid2020on2021geography.xls"

pop_data = pd.read_excel(population_filename, sheet_name="MYE4", skiprows=7, 
                         usecols=["Code", "Name", "Mid-2020"])

# Using rename method to update column names
pop_data = pop_data.rename(columns={"Name":"county", "Mid-2020":"county_population"})
pop_data

ImportError: Missing optional dependency 'xlrd'. Install xlrd >= 1.0.0 for Excel support Use pip or conda to install xlrd.

For ease later on, we have renamed the columns, including the column containing the county (and other) values to "county" to match to our UKPVGeo data set (using the [`rename` method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html))

---

### Exercise B

For the modified UKPVGeo database, we can now select by county rather than having to define a rough area as we did before.

*For those interested, details of how the county data was added to our UKPVGeo database are available in [Workshop2_Extra_SpatialJoin](Workshop2_Extra_SpatialJoin.ipynb). This used something called a spatial join, but this is **not** part of the learning objective for this session.*

For use in this exercise, we will start by grouping the solar panel data by county, and finding the sum of each column.

In [41]:
# Group the ukpvgeo by the "county" column, finding the sum for each group
# The reset_index() menthod is used to keep "county" as a column rather than setting this as the new index (for now)

ukpvgeo_sum = ukpvgeo.groupby("county").sum().reset_index()
ukpvgeo_sum

C:\Users\petes\AppData\Local\Temp\ipykernel_1928\3934210031.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ukpvgeo_sum = ukpvgeo.groupby("county").sum().reset_index()


,county,capacity_repd_MWp,capacity_osm_MWp,area_sqm
0,Barking and Dagenham,0.0,0.032,11838.8
1,Barnet,0.0,0.012,415.3
2,Barnsley,1.7,0.000,1362.1
3,Bath and North East Somerset,7.3,7.340,155940.1
4,Bedford,73.8,54.300,1975707.8
...,...,...,...,...
169,Wokingham,5.0,5.000,108715.9
170,Wolverhampton,0.0,0.000,61984.0
171,Worcestershire,87.8,65.251,1914269.4
172,Wrexham,51.6,29.800,938484.5


We now want to merge this database with the population data to allow us to look at the total listed capacity of solar panels by population for each county. 

1) For the `ukpvgeo_sum` and `pop_data` DataFrames consider how you can merge them. What is the common column and what type of join do you want to use?

2) Calculate the capacity per unit of population for the county data. Which county has the highest value? 

In [16]:
### ADD CODE HERE


3) Save your new merged DataFrame out as an SQLite database ([Activity 1 notes](Workshop2_01_Databases.ipynb#Saving-to-a-database))

In [17]:
### ADD CODE HERE


---